In [1]:
import torch
import torchvision.models as models
import torch.nn as nn

# 사전 학습된 ResNet-50 모델 불러오기
model = models.resnet50(pretrained=True)

# 마지막 FC 레이어 제거하고, 특징 벡터를 출력하도록 수정
model = nn.Sequential(*list(model.children())[:-1])

# 모델을 평가 모드로 전환
model.eval()


c:\Users\user\anaconda3\envs\torch-book\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user\anaconda3\envs\torch-book\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [3]:
from torchvision import transforms
from PIL import Image

# 이미지 전처리 파이프라인 정의
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 예제 이미지 불러오기 및 전처리
img = Image.open("path_to_image.jpg")
img_tensor = preprocess(img).unsqueeze(0)  # 배치 차원 추가

# 특징 벡터 추출
with torch.no_grad():
    features = model(img_tensor)
    features = features.view(features.size(0), -1)  # 벡터 형태로 변환

print("Extracted feature vector shape:", features.shape)


Extracted feature vector shape: torch.Size([1, 2048])


In [4]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# 레이블링 데이터 (예: [0, 1, 2, ...] 형태의 레이블) 로드
labels = torch.tensor([0])  # 예제에서는 하나의 레이블만 사용

# 특징 벡터와 레이블을 데이터셋으로 만듦
dataset = TensorDataset(features, labels)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# 간단한 선형 분류기 정의
classifier = nn.Linear(features.size(1), 10)  # 출력 클래스 수에 맞게 조정 (예: 10 클래스)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# 분류기 학습
classifier.train()
for epoch in range(10):  # 예제: 10 에폭 동안 학습
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = classifier(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

print("Training completed.")


Training completed.
